# Générateurs

Le concept de _générateur_ Python est associé à l'idée d'une exécution "paresseuse" du code. L'idée principale d'un générateur est de générer le contenu d'une liste _à la demande_, plutôt qu'en une seule fois. On peut illustrer l'aspect "paresseux" avec un exemple utilisant ```range```:

In [1]:
def premier_element(liste):
    return liste[0]

In [2]:
range(5000)

range(0, 5000)

Conceptuellement, ```range(5000)``` représente la liste [0, 1, 2... 4999], mais techniquement on a ici un object ```range``` qui ne contient pas la liste complète, seulement les paramètres nécessaires pour la générer. 

In [3]:
premier_element(range(5000))

0

Le résultat est identique à celui qu'on aurait obtenu en générant la liste complète [0,...4999] _avant_ d'en prendre le premier élément, mais l'opération effectuée ici a été d'appliquer l'opération ```[0]``` (premier élément) à un générateur, soit initialiser la séquence, générer le premier élément, et s'arrêter.

On a donc évité un gros travail inutile: on utilise le terme "paresseux", mais on a en réalité un gain d'efficacité et une économie de la mémoire, notamment parce que la séquence complète n'est jamais matérialisée en mémoire (à moins de le demander explicitement, comme par exemple en créant la liste:

In [4]:
list(range(20))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

## Lien avec l'évaluation paresseuse en lambda calcul
L'expression ```premier_element(range(5))``` est de la forme ```f(g(x))```, une expression qui se traduirait en lambda calcul par un terme avec deux redex: une application dont le sous-terme de droite serait une autre application, et ces deux applications formant chacune un redex. Le terme pourrait être réduit par une séquence de deux beta-réductions, en commençant par l'un ou l'autre des redex. 

La stratégie _applicative_ consisterait à réduire d'abord l'application ```g(x)```, puis à réduire dans une deuxième étape l'application de ```f``` au résultat. La stratégie d'évaluation _paresseuse_ consisterait elle à réduire directement l'application extérieure ```f(...)```, c'est-à-dire écrire le corps de la fonction ```f``` en substituant le paramètre par ```g(x)```, et dans la deuxième étape effectuer la réduction correspondant à l'application ```g(x)```. 

Ici, comme dans une stratégie paresseuse du lambda calcul, on substitue l'objet ```range``` dans le corps de la fonction ```premier_element```, et on se retrouve avec l'expression ```range(5000)[0]```, qui peut être gérée plus efficacement, car on sait qu'on a seulement besoin de déterminer le premier élément de la liste. 

## Générateurs de la bibliothèque Python

La fonction ```range``` est sans doute le générateur le plus utilisé en Python (même si un ```range``` n'est pas en fait une instance de la classe ```generator```, leur comportements sont similaires). Plusieurs autres fonctions de la bibliothèque standard Python qui produisent des listes en sortie sont aussi des générateurs, notamment ```zip```, ```map```, et ```filter```.

In [5]:
zip([3, 4, 5], ["a", "b", "c"])

Comme c'était le cas avec ```range```, les éléments de la liste n'ont pas encore été calculés: ils le seront à la demande, comme par exemple en exécutant une boucle ```for```:

In [6]:
listez = zip([3, 4, 5], ["a", "b", "c"])
for z in listez:
    print(z)

(3, 'a')
(4, 'b')
(5, 'c')


Les compréhensions de liste peuvent être écrites sous forme de générateurs: il suffit de remplacer les crochets par des parenthèses:

In [7]:
[a*a for a in [3, 5, 8]]

[9, 25, 64]

In [8]:
(a*a for a in [3, 5, 8])

<generator object <genexpr> at 0x10ba38e40>

Comme pour les autres générateurs, on peut obtenir le contenu de la liste à la demande:

In [9]:
g = (a*a for a in [3, 5, 8])

In [10]:
list(g)

[9, 25, 64]

## Utilisation de générateurs

Comme on l'a montré ci-dessus, on peut énumérer les éléments générés à l'aide d'une boucle ```for```: ceci s'applique à ```range``` comme aux autres fonctions mentionnées, et aux compréhensions de listes. 

In [11]:
g = (a*a for a in [3, 5, 8])

In [12]:
n =0
for i in g:
    n = n+1
    print(i)
print(n, "élements")

9
25
64
3 élements


Cependant, pour tous les générateurs autres que ```range```, les éléments sont générés puis "oubliés", et par conséquent on ne peut énumérer les éléments qu'une seule fois. Ceci peut être un problème dans un algorithme qui requiert de parcourir une liste plusieurs fois (tri...). 

In [13]:
n =0
for i in g:
    n = n+1
    print(i)
print(n, "élements")

0 élements


Pour bien comprendre ce qu'il se passe, il faut savoir que l'énumération des valeurs, que ce soit dans une boucle ```for``` ou pour construire une liste, utilise en arrière-plan une fonction ```next()```, qui permet de calculer la prochaine valeur:

In [14]:
h = (a for a in [57, 81])

In [15]:
next(h)

57

In [16]:
next(h)

81

Et le processus d'énumération se termine avec un événement (une sorte d'exception) ```StopIteration```:

In [17]:
next(h)

StopIteration: 

À ce point, on est à la fin de l'énumération et on ne peut plus accéder aux valeurs précédentes.

Ce comportement "à état" peut causer des bugs assez subtils. Prenons le générateur suivant, qui génère les valeurs [1, 2, 3, 4]:

In [18]:
gen = (a for a in [1, 2, 3, 4])

Si on oublie qu'on est en présence d'un générateur et non pas d'une liste, on peut essayer de vérifier si une valeur est dans la liste, à l'aide de l'opérateur ```in```:

In [19]:
2 in gen

True

Jusqu'ici, tout va bien: 2 est bien dans la liste. Mais pour évaluer si le nombre 2 était dans la liste, la mécanique en arrière de l'opérateur ```in``` a énuméré les valeurs jusqu'à trouver 2. L'état du générateur est donc "arrêté" après le 2. 

In [20]:
3 in gen

True

C'est toujours correct, puisque 3 est bien dans la liste. Mais c'est une chance: on s'était arrêté après le 2, et on a repris à cet endroit, ce qui nous a permis de trouver le 3.  

In [21]:
1 in gen

False

Cette fois, on a un problème: 1 est dans la liste, maisn'est trouvée dans l'énumeration. Ceci s'explique par le fait que l'énumération a continué après le 3, et il restait donc seulement le 4. La valeur 1 n'a donc pas été trouvée.

In [22]:
4 in gen

False

Encore une fois l'énumération a continué, mais en cherchant le 1 on était arrivé au bout de la liste, et à présent on ne trouve même plus le 4, alors que c'était le dernier élément. À ce point-ci, plus aucune valeur ne sera trouvée comme étant ```in gen```.

Notons que ces problèmes ne se posent pas avec ```range```, et ceci s'explique par le fait que la classe ```range``` est différente de la classe générateur (```generator```): aucune n'est sous-classe de l'autre. Avec ```range```, on doit accéder à un itérateur associé à l'objet en invoquant ```iter(...)```, et l'énumération recommence à chaque fois qu'on crée un nouvel itérateur (ce qui n'est pas le cas pour des générateurs).

In [23]:
k = range(5)
n =0
for i in k:
    n = n+1
    print(i)
print("===",n, "éléments")

n =0
for i in k:
    n = n+1
    print(i)
print("== encore", n, "éléments")

0
1
2
3
4
=== 5 éléments
0
1
2
3
4
== encore 5 éléments


La classe ```range``` permet aussi d'utiliser l'opérateur ```[]``` pour accéder à un ou plusieurs éléments de l'intervalle:

In [24]:
k[1]

1

In [25]:
k[2:-1]

range(2, 4)

## Définir son propre générateur

On peut facilement créer un générateur qui ne soit pas une compréhension de liste ou une fonction existante. Le plus simple est de créer une fonction qui construit la liste de valeurs à retourner, mais au lieu de les stocker effectivement dans une liste, chaque valeur est renvoyée à l'aide de ```yield```: tout se passe comme si on faisait un ```return``` de chaque valeur de la liste, mais sans réellement quitter la fonction. Typiquement, on place donc le ```yield``` à l'intérieur d'une boucle. Par exemple, on peut implémenter comme suit un générateur qui énumère les valeurs de n à 0:

In [26]:
def compte_a_rebours(n):
    while(n>0):
        n -=1
        yield n

Remarquer l'absence de return et l'absence de stockage des valeurs. Testons:

In [27]:
for j in compte_a_rebours(7):
    print(j)

6
5
4
3
2
1
0


Utilisons ```next()```:

In [28]:
g = compte_a_rebours(3)

In [29]:
next(g)

2

In [30]:
next(g)

1

In [31]:
next(g)

0

In [32]:
next(g)

StopIteration: 

On obtient donc un objet avec les fonctionnalités de base d'un générateur.

Il est aussi pertinent de montrer que la présence d'une boucle n'est pas indispensable, et le yield n'est pas nécessairement unique:

In [33]:
def mon_gen():
    yield 3
    print("hello")
    yield 'w'
    print("dernier element!")
    yield 0

In [34]:
a = mon_gen()
next(a)

3

In [35]:
next(a)

hello


'w'

In [36]:
next(a)

dernier element!


0

In [37]:
next(a)

StopIteration: 

#### Exercice 1

Écrire un générateur qui prend une chaine de caractères en entrée et énumère les caractères la composant